# Decision Trees Implementation

## Submitted by: Hardik Garg, part of CN DS+ML course

Note to the grader: The optional part is also done (actual decision tree is implemented), which means that "printing decision tree steps as in example" and actual decision tree implementation" are done together.

In other words, calling the fit function on the decision_tree prints the levels as in example and trains the model, both at the same time.

Then, PDFs for both - the iris dataset and the OR logic gate are saved.

In [245]:
import numpy as np, pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
import pydotplus
import math

In [246]:
class Node:
    
    def __init__(self, split_feature, majority_class):
        
        ## split_feature represents feature by which we split current node
        self.split_feature = split_feature
        
        ## dictionary to hold <feature_value> - <child_node>
        self.children = {}
        
        ## store the majority class corresponding to the current node
        self.majority_class = majority_class
        
        ## index (for keeping track) of current node
        self.index = -1
        
    def add_child(self, feature_value, child):
        
        self.children[feature_value] = child
    
    

In [247]:
class Decision_Tree:
    
    def __init__(self):
        
        ## root of our tree is initially None
        self.root=None
        
    ## returns a dictionary corresponding to the freq of each class in y
    def get_class_freq(self, y):
        
        freq_dict = {}
        
        for val in y:
            
            if val in freq_dict:
                freq_dict[val]+=1
            else:
                freq_dict[val]=1
        
        return freq_dict
       
    ## returns entropy of given array
    def entropy(self, y):
        class_freq_dict = self.get_class_freq(y)
        _entropy=0
        total_num = len(y)
        
        for key in class_freq_dict:
            
            p = class_freq_dict[key]/total_num
            _entropy += -p*math.log2(p)
            
        return _entropy
    
    ## function to return the gain ratio
    def gain_ratio(self, x, y, feature):
        
        ## entropy before splitting
        info_gain_parent = self.entropy(y)
        
        ## entropy, split_info after splitting is calculated now
        info_gain_children = 0
        split_info = 0
        
        ## extract unique x values
        unique_values = set(x[:,feature])
        
        ## dataframe having x-y values
        df = pd.DataFrame(x)
        df[df.shape[1]] = y
        
        ## total number of all values of the feature (denominator)
        total_num = df.shape[0]
        for val in unique_values:
            
            df_curr = df[df[feature]==val]
            
            ## total entries in current val of feature (numerator)
            curr_num = df_curr.shape[0]
            
            ## calling entropy on the last col of dataframe
            ## we are taking weighted sum of entropies of all values of the feature
            info_gain_children += (curr_num/total_num)*self.entropy(df_curr[df_curr.shape[1]-1])
            
            ## update split info
            if curr_num != 0:
                split_info += -(curr_num/total_num)*math.log2(curr_num/total_num)
        
        ## when split_info is 0, we have a pure node
        if split_info == 0:
            return math.inf
        
        info_gain = info_gain_parent - info_gain_children
        return info_gain / split_info
            
            
        
    ## function to build tree, level denotes current depth, all_feature_names stored names of all features
    ## (useful for printing)
    ## this is a recursive function
    ## returns the root of the decision tree
    def build_decision_tree(self, x, y, features, level, all_classes, all_feature_names=np.array([f for f in df.columns])):
        
        ## base case
        
        ## if no features are left
        if len(features)==0:
            
            majority_class = None
            class_freq_dict = self.get_class_freq(y)
            max_freq_count = -math.inf
            
            print("Level:",level)
            
            for _class in all_classes:
                
                if _class in class_freq_dict:
                    
                    if class_freq_dict[_class] > max_freq_count:
                        max_freq_count = class_freq_dict[_class]
                        majority_class=_class
                        
                    print("Count of", _class,":", class_freq_dict[_class])
                    
                else:
                    print("Count of", _class,":",0) 
                    
            print("Current Entropy: ", self.entropy(y))
            print("Reached leaf node")
            print()
            
            ## leaf node can't be split further so there is no split_feature, but there is a majority_feature
            return Node(None, majority_class)
        
        
        ## if the node is pure
        if len(set(y)) == 1:
            
            majority_class = None
            
            print("Level:",level)
            
            for _class in all_classes:
                
                if _class in y:
                    majority_class = _class
                    print("Count of", _class,":", len(y))
                    
                else:
                    print("Count of", _class,":",0) 
                    
            print("Current Entropy: ", 0)
            print("Reached leaf node")
            print()
            
            ## leaf node can't be split further so there is no split_feature, but there is a majority_feature
            return Node(None, majority_class)
        
        
        ## now we write the main part which is run when base case is not executed. 
        ## here we find the best_split_feature to split the current node (generating maximum information gain)
        
        best_split_feature = None
        max_gain_ratio = -math.inf
        
        for feature in features:
            
            curr_gain_ratio = self.gain_ratio(x,y,feature)
            
            if curr_gain_ratio > max_gain_ratio:
                
                max_gain_ratio = curr_gain_ratio
                best_split_feature = feature
                
                
        ## printing details for current level
        
        ## finding majority_class in current y
        class_freq_dict = self.get_class_freq(y)
        max_freq_count = -math.inf
        majority_class=None
        
        print("Level:",level)
            
        for _class in all_classes:
                
            if _class in class_freq_dict:
                    
                if class_freq_dict[_class] > max_freq_count:
                    max_freq_count = class_freq_dict[_class]
                    majority_class=_class
                        
                print("Count of", _class,":", class_freq_dict[_class])
                    
            else:
                print("Count of", _class,":",0) 
                    
        print("Current Entropy: ", self.entropy(y))
        print("Splitting on feature", all_feature_names[best_split_feature], "with gain ratio:", max_gain_ratio)
        print()
        
        ## preparing for recursive calls
            
        ## getting set of all unique x values
        unique_values = set(x[:,best_split_feature])
        
        df = pd.DataFrame(x)
        ## append y value in the end 
        df[df.shape[1]]=y
        
        ## creating proper tree node (root)
        root_node = Node(best_split_feature, majority_class)
        
        ## remove best_split_feature 
        idx = features.index(best_split_feature)
        features.remove(best_split_feature)
        #idx = np.where(features==best_split_feature)
        #features = np.delete(features, idx)
        
        ## calling recursion on all children of root_node
        
        for val in unique_values:
            
            ## dataFrame corresponding to current unique value, using boolean indexing
            df_curr = df[df[best_split_feature]==val]
            
            ## defining x_ and y_ for the child
            x_ = df_curr.iloc[:,:df_curr.shape[1]-1].values
            y_ = df_curr.iloc[:, df_curr.shape[1]-1].values
            
            child_node = self.build_decision_tree(x_, y_, features, level+1, all_classes)
            root_node.add_child(val, child_node)
            
        ## re-add the removed feautre
        features.insert(idx, best_split_feature)
        #features.insert(best_split_feature, idx)
        #np.insert(features, idx, best_split_feature)
        
        ## return root_node
        return root_node
        
        
    ## function to fit dataset to decision tree
    def fit(self, x, y):
        
        ## ex - for iris, x has 4 features which is len(x[0])
        features = [i for i in range(len(x[0]))]
        
        ## all_classes includes the "names" of all output classes
        all_classes = set(y)
        level=0
        
        ## fitting the model
        self.root = self.build_decision_tree(x,y,features,level,all_classes)
            
    ## recursive function to reach the result for a single training point
    ## returns the majority class accordingly
    
    def single_point_predict(self, xi, node):
        
        ## base case (when we are at a leaf node)
        if len(node.children) == 0:
            return node.majority_class
        
        ## value of xi corresponding to the split feature 
        split_feature_value=xi[node.split_feature]
        
        ## if current class is a pure class
        if split_feature_value not in node.children:
            return node.majority_class
        
        ## else call recursion
        return self.single_point_predict(xi, node.children[split_feature_value])
        
    def predict(self, x):
        
        y = np.array([0 for i in range(len(x))])
        
        for i in range(len(x)):
            y[i] = self.single_point_predict(x[i], self.root)
        
        return y
       
    def score(self, x, y):
        
        ## returns the mean accuracy
        y_pred = self.predict(x)
        count=0
        
        for i in range(len(y)):
            
            if y[i] == y_pred[i]:
                count+=1
                
        return count/len(y)
        
    def export_tree_pdf(self, filename=None):
        
        from collections import deque
        
        dot_data = '''digraph Tree {node [shape=box] ;'''
        
        queue = deque()
        
        r = self.root
        queue.append(r)
        count = 0
        if r.index == -1:
            r.index = count
        
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.split_feature,r.majority_class) 
        
        # Doing LEVEL ORDER traversal in the tree (using a queue)
        while len(queue) != 0 :
            node = queue.popleft()
            for i in node.children:
                count+=1
                if(node.children[i].index==-1):
                    node.children[i].index = count
                
                # Creating child node
                dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nMajority_Class at this node : {}\" ];".format(node.children[i].index,node.children[i].split_feature,node.children[i].majority_class) 
                # Connecting parent node with child
                dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
                # Adding child node to queue
                queue.append(node.children[i])
        
        dot_data = dot_data + "\n}"

        if filename != None:    
            graph = pydotplus.graph_from_dot_data(dot_data)
            graph.write_pdf(filename)    
        
        return dot_data
                    
                
                
        
        

In [248]:
## some data preprocessing
iris = datasets.load_iris()

df = pd.DataFrame(iris.data)

## sepal_length, sepal_width, petal_length, petal_width
df.columns = ["sl", "sw", "pl", "pw"]

## function to convert values in each field to discrete values based on a threshold
## we take 4 boundaries as discussed in lecture video around 3 points-
## 1. (min_val + mean)/2 => this is less than mean
## 2. mean_value => this is equal to mean
## 3. (max_value + mean)/2 => this is greater than mean

def convert_to_discrete(value, *args):
    
    if value < args[0]:
        return 0
    if value < args[1]:
        return 1
    if value < args[2]:
        return 2
    return 3
    
for col in df.columns:
    
    mean_val = df[col].mean()
    min_val = df[col].min()
    max_val = df[col].max()
    
    ## array holding points around which we split continuous values into labels
    threshold_arr = np.array([(min_val+mean_val)/2, mean_val, (max_val+mean_val)/2])
    
    df[col] = df[col].apply(convert_to_discrete, args=(threshold_arr[0], threshold_arr[1], threshold_arr[2]))

In [249]:
## testing the decision tree

x = df.values
y = iris.target
features = set(df.columns)
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1)

In [250]:
## build tree

dec_tree = Decision_Tree()

## fit
dec_tree.fit(x_train, y_train)

## saving pdf
dec_tree.export_tree_pdf(filename = "iris_tree.pdf")

Level: 0
Count of 0 : 37
Count of 1 : 34
Count of 2 : 41
Current Entropy:  1.5807197138422104
Splitting on feature pl with gain ratio: 0.6802317879850113

Level: 1
Count of 0 : 37
Count of 1 : 0
Count of 2 : 0
Current Entropy:  0
Reached leaf node

Level: 1
Count of 0 : 0
Count of 1 : 6
Count of 2 : 0
Current Entropy:  0
Reached leaf node

Level: 1
Count of 0 : 0
Count of 1 : 28
Count of 2 : 17
Current Entropy:  0.9564574047992594
Splitting on feature pw with gain ratio: 0.35184707702123014

Level: 2
Count of 0 : 0
Count of 1 : 3
Count of 2 : 0
Current Entropy:  0
Reached leaf node

Level: 2
Count of 0 : 0
Count of 1 : 25
Count of 2 : 8
Current Entropy:  0.7990485210442682
Splitting on feature sl with gain ratio: 0.14662562110534788

Level: 3
Count of 0 : 0
Count of 1 : 0
Count of 2 : 1
Current Entropy:  0
Reached leaf node

Level: 3
Count of 0 : 0
Count of 1 : 7
Count of 2 : 0
Current Entropy:  0
Reached leaf node

Level: 3
Count of 0 : 0
Count of 1 : 16
Count of 2 : 7
Current Entropy

'digraph Tree {node [shape=box] ;\n0 [label="Feature to split upon : X[2]\\nOutput at this node : 2" ];\n1 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 0" ];\n0 -> 1 [ headlabel="Feature value = 0"]; \n2 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 1" ];\n0 -> 2 [ headlabel="Feature value = 1"]; \n3 [label="Feature to split upon : X[3]\\nMajority_Class at this node : 1" ];\n0 -> 3 [ headlabel="Feature value = 2"]; \n4 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 2" ];\n0 -> 4 [ headlabel="Feature value = 3"]; \n5 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 1" ];\n3 -> 5 [ headlabel="Feature value = 1"]; \n6 [label="Feature to split upon : X[0]\\nMajority_Class at this node : 1" ];\n3 -> 6 [ headlabel="Feature value = 2"]; \n7 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 2" ];\n3 -> 7 [ headlabel="Feature value = 3"]; \n8 [label="Feature to split upon :

In [251]:
print("Train score: ",dec_tree.score(x_train, y_train))
print("Test score: ",dec_tree.score(x_test, y_test))

Train score:  0.9375
Test score:  1.0


In [252]:
## generating pdf for OR gate

dec_tree = Decision_Tree()

x_or = np.array([[0,0], [0,1], [1,0], [1,1]])
y_or = np.array([0,1,1,1])

dec_tree.fit(x_or, y_or)
y_pred = dec_tree.predict(x_or)

## saving pdf
dec_tree.export_tree_pdf(filename = "or_tree.pdf")


Level: 0
Count of 0 : 1
Count of 1 : 3
Current Entropy:  0.8112781244591328
Splitting on feature sl with gain ratio: 0.31127812445913283

Level: 1
Count of 0 : 1
Count of 1 : 1
Current Entropy:  1.0
Splitting on feature sw with gain ratio: 1.0

Level: 2
Count of 0 : 1
Count of 1 : 0
Current Entropy:  0.0
Reached leaf node

Level: 2
Count of 0 : 0
Count of 1 : 1
Current Entropy:  0.0
Reached leaf node

Level: 1
Count of 0 : 0
Count of 1 : 2
Current Entropy:  0
Reached leaf node



'digraph Tree {node [shape=box] ;\n0 [label="Feature to split upon : X[0]\\nOutput at this node : 1" ];\n1 [label="Feature to split upon : X[1]\\nMajority_Class at this node : 0" ];\n0 -> 1 [ headlabel="Feature value = 0"]; \n2 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 1" ];\n0 -> 2 [ headlabel="Feature value = 1"]; \n3 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 0" ];\n1 -> 3 [ headlabel="Feature value = 0"]; \n4 [label="Feature to split upon : X[None]\\nMajority_Class at this node : 1" ];\n1 -> 4 [ headlabel="Feature value = 1"]; \n}'

# The End :)